<div class="alert alert-block alert-info">
<h1>Correlation between Oil and Fuel Prices in Portugal</h1>
<h2>FCD 1st intermediate project presentation</h2>
<h3>Rodrigo Pinto 103280 </h3>
</div>

Oil is one of the most important and widely used commodities in the world, since it is the primary global energy source. Derived from fossilized organic matter that accumulated in geological layers over millions of years, crude oil is refined into products such as gasoline, diesel, kerosene, fuel oil, and other petrochemical derivatives, which are essential for various industries, transportation, and energy generation.

The evolution of oil prices has been marked by highs and lows, with several economic, political, and environmental crises playing an important role.

The rise in oil prices directly affects the prices of fuels such as gasoline and diesel. However, it is important to understand that taxation also has an impact on the final price.

***

<div class="alert alert-block alert-info">
<h2>State of the art</h2>
</div>

In every project, is mandatory doing an initial research. Therefore, I found a brief description about the work of the 2nd place of last year Arquivo.pt awards, which was Habitação.NET.

The main goal of this project is to allow the user to explore the evolution of the housing and rental market in Portugal. Besides, the author of this project used as source of information of about 30 million adds from different real estate web pages and 15 thousand news stored in arquivo.pt since 2001. Additionally, Habitação.NET offers the possibility to analyse the price trends at a national level as well as between districts, municipalities and parishes.

The data retrieving and processing methodology that the author employed in his project is the following:

- ##### Listing real estate web browsers 
    Use of manual search in Arquivo.pt with API Full-text Search
	<p>Research on Marktest platform</p>
<br></br>
- ##### Identify the adds
    Mapping the structure of each platform's pages
	<p>Finding the relevants adds</p>
	Processed pages : 32 million by using API CDX of Arquivo.pt
<br></br>
- ##### Retrieve and Analyse the data
    Download the proccessed pages by using API Wayback from arquivo.pt
	<p>FYI it was about 296GB of data</p>
	RE parameter extracte: Price, Area, Type, Location and description
<br></br>
- ##### Unify the data
    Each RE platform uses different nomenclature for the same type of information, so there was a need to normalize it by tuning the 	data into a unified format.
<br></br>
- ##### Extract the location of each real estate
    There was a lack of consistency about the actual place of each piece of real estate betwween plataforms. So the author used the maps published by the Direção Geral do Território and the OpenStreetMap Nominatim API for the mapping of locations.
<br></br>
- ##### Filter the data
    Commercial and industrial places were discarded from the data set.
<br></br>
- ##### Calculate the average price of the m2 



***

<div class="alert alert-block alert-info">
<h2>Methodology</h2>
</div>

As mentioned before, the main goal is to do a correlation between the oil prices and fuel prices in Portugal. In order to accomplish that, it is crucial to gather the right information. 


***

<div class="alert alert-block alert-info">
<h3>Initial Approach</h3>
</div>

Initially, I analysed all the web pages stored in the arquivo.pt related to fuel prices and I came up with 2 predominants web pages (compareomercado, maisgasolina). Although I didn't find any web page related to oil prices, I seacrhed outside arquivo.pt and I discovered this web page https://www.kaggle.com/datasets/mabusalah/brent-oil-prices that allows the user to download a csv file with a data set that was retrieved from the U.S Energy Information Administration: Europe Brent Spot Price (Dollars per Barrel) that contains the oil price since 1987.

In [3]:
import requests
import nltk
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def extract(dates, link_url):
    html = ""         
    
    
    #extract dates
    datas = dates['tstamp']
    y = datas[0:4]
    m = datas[4:6]
    d = datas[6:8]
    h = datas[8:10]
    min = datas[10:12]
    s = datas[12:]

    final_date = y+"-"+m+"-"+d
    print(f"{d}/{m}/{y} at {h}:{min}:{s}")

    #extract the link to the text
    original_url = dates['linkToExtractedText']
    link_url.append(original_url)


    return link_url, final_date

In [ ]:

topics = ["maisgasolina.com", "compareomercado.pt/precos-combustiveis-gasolina-gasoleo"]
items = 5000
begin =20080218203700
last =20230217011100
url_link_to_extract = []
link_url = []
title_list = []
prefinal_date_list = []
final_date_list = []
date_rearrange = []
for topic in topics:
    print(f"Topic: {topic}\n")

    url = f"https://arquivo.pt/textsearch?q={topic}&maxItems={items}&prettyPrint=true&from={begin}&to={last}"
    r = requests.get(url)

    if r.status_code == 200:
        print("Success!")
        data = r.json()
        print(data)
        a = 0
        for dates in data['response_items']:
            title = dates['title']
            a+=1
            if len(title) != 0 and not title.__contains__("301 Moved Permanently") and (title.__contains__('Mais Gasolina') or title.__contains__('Compare o Mercado')):                                                                       
                print("PASS")
                url_link_to_extract, dates_4_file = extract(dates, link_url)                    
                final_date_list.append(dates_4_file)
                title_list.append(title)
            
            else:
                print("FAIL")
    else:
        print("Something is wrong")

Topic: maisgasolina.com

Success!
{'serviceName': 'Arquivo.pt - the Portuguese web-archive', 'linkToService': 'https://arquivo.pt', 'estimated_nr_results': 98540, 'request_parameters': {'offset': 0, 'dedupValue': 2, 'from': '20080218203700', 'to': '20230217011100', 'dedupField': 'site', 'q': 'maisgasolina.com', 'maxItems': 500}, 'response_items': [{'title': '301 Moved Permanently', 'originalURL': 'http://www.maisgasolina.com/', 'linkToArchive': 'https://arquivo.pt/wayback/20140905173413/http://www.maisgasolina.com/', 'tstamp': '20140905173413', 'contentLength': 237, 'digest': 'cfa7890c3a73ba9ff902c7f9b980c041', 'mimeType': 'text/html', 'encoding': 'windows-1252', 'date': '1409938453', 'linkToScreenshot': 'https://arquivo.pt/screenshot?url=https%3A%2F%2Farquivo.pt%2FnoFrame%2Freplay%2F20140905173413%2Fhttp%3A%2F%2Fwww.maisgasolina.com%2F', 'linkToNoFrame': 'https://arquivo.pt/noFrame/replay/20140905173413/http://www.maisgasolina.com/', 'linkToExtractedText': 'https://arquivo.pt/textextr

In [5]:
##Sorting the title of each browser link to extracted text by the date 
ziped = list(zip(final_date_list, url_link_to_extract, title_list))
ziped.sort()
final_date_list, url_link_to_extract, title_list = zip(*ziped)
final_date_list = list(final_date_list)
url_link_to_extract = list(url_link_to_extract)
title_list = list(title_list)

#Creating a dictionary
dict_search = {'Title':title_list, 'Link To Extracted Text': url_link_to_extract, 'Date': final_date_list}

#Creating a csv file
dt_search = pd.DataFrame(data = dict_search)
dt_search.to_csv('Web_search', index=False)

In [7]:
print(dt_search.head())

                                               Title  \
0  Mais Gasolina - Preço dos combustíveis em Port...   
1  Mais Gasolina - Preço dos combustíveis em Port...   
2  Mais Gasolina - Preço dos combustíveis em Port...   
3  Postos com o combustível mais barato | Mais Ga...   
4  Mais Gasolina - Preços dos combustíveis em Por...   

                              Link To Extracted Text        Date  
0  https://arquivo.pt/textextracted?m=http%3A%2F%...  2008-02-18  
1  https://arquivo.pt/textextracted?m=http%3A%2F%...  2008-10-22  
2  https://arquivo.pt/textextracted?m=http%3A%2F%...  2009-06-24  
3  https://arquivo.pt/textextracted?m=http%3A%2F%...  2009-06-24  
4  https://arquivo.pt/textextracted?m=http%3A%2F%...  2009-09-26  


<div class="alert alert-block alert-info">
<h3>Current state of the project</h3>
</div>

At the moment, the project is in the data filtering stage. This stage is crucial as it discards irrelevant information. Therefore, I analyzed the characteristics of the websites' structure over the years to highlight patterns in order to extract information effectively.

In [ ]:
stop_words = set(stopwords.words('portuguese'))


with open("C:\\Users\\ASUS\\Documents\\GitHub\\tutorial\\web_search_1", 'r') as f:
    df = pd.read_csv(f, delimiter= ',')


list_1,list_2, list_3, list_4, list_5, final_list = [], [], [], [], [], []

list_dates_1,list_dates_2,list_dates_3,list_dates_4,list_dates_5, final_list_dates = [], [], [], [], [], []


In [ ]:
def remove_words(doc):
    return [word for word in doc if word.lower() not in stop_words]        


In [ ]:
def numeric(f_doc):
    str_word = ""
    list_price = []
    price_pattern = r"\d{1}[.,]\d{3}"  #padrao do preço \b corresponde à ancora (limite da palavra) \d{1} um numero decimal [.,] ponto e virgula \d{3} tres numeros decimais
    for word in f_doc:
        str_word += word
    price = re.findall(price_pattern, str_word)
    return price

In [ ]:
def fuel(g, fuel_text):
    lista_debug_1, lista_debug_1_, lista_debug_2, lista_debug_2_, lista_debug_2_a, lista_debug_2_aa, lista_debug_3, lista_debug_4, lista_debug_4_date, lista_debug_5, lista_debug_5_date = [], [], [], [], [], [], [], [], [], [], []
    i_1 =  i_2 =  i_3 = i_4 = i_5 = 0
    if g == 1:
        for word in fuel_text:
            i_1+=1
            if  word.lower() == "gasolina":
                word_n= fuel_text[i_1]
                lista_debug_1.append(word_n)
                lista_debug_1_ = numeric(lista_debug_1)
                for i in range(len(lista_debug_1_)):
                    lista_debug_1_[i] = float(lista_debug_1_[i])
                av = round(np.mean(lista_debug_1_),3)
                lista_debug_1_.clear()
                lista_debug_1_.append(av)
        return lista_debug_1_
    elif g ==2 :
        for word in fuel_text:
            i_2+=1
            if  word == "95":
                word_n= fuel_text[i_2+1]
                lista_debug_2.append(word_n)
                lista_debug_2_ = numeric(lista_debug_2)
                for i in range(len(lista_debug_2_)):
                    lista_debug_2_[i] = float(lista_debug_2_[i])
                av = round(np.mean(lista_debug_2_),3)
                lista_debug_2_.clear()
                lista_debug_2_.append(av)
                
        return  lista_debug_2_
    elif g == 3:
        for word in fuel_text:
            i_3+=1
            if  word == "Repsol" :#and fuel_text[i_3-1] == "/":
                word_1= fuel_text[i_3]
                word_2 = fuel_text[i_3+5]
                word_3 = fuel_text[i_3+10]
                i_3 = 0

                lista_debug_2_a.append(word_1)
                lista_debug_2_a.append(word_2)
                lista_debug_2_a.append(word_3)

                lista_debug_2_aa = numeric(lista_debug_2_a)
                for i in range(len(lista_debug_2_aa)):
                    lista_debug_2_aa[i] = float(lista_debug_2_aa[i])
                av = round(np.mean(lista_debug_2_aa),3)
                lista_debug_2_aa.clear()
                lista_debug_2_aa.append(av)
        return  lista_debug_2_aa
    elif g==4:
        for word in fuel_text:
            i_4+=1
            if word.lower() == "gasolina" and fuel_text[i_4].lower() == "especial" and fuel_text[i_4+1] == "95":
                word_4 = fuel_text[i_4+2]
                word_4_date = fuel_text[i_4+3]
                lista_debug_4.append(word_4)
                lista_debug_4 = numeric(lista_debug_4)
                lista_debug_4_date.append(word_4_date)
        return lista_debug_4, lista_debug_4_date
    elif g==5:
        for word in fuel_text:
            i_5+=1
            if word.lower()=="gasolina" and fuel_text[i_5]=="95":
                word_5 = fuel_text[i_5+1]
                word_5_date = fuel_text[i_5+2]
                lista_debug_5.append(word_5)
                lista_debug_5 = numeric(lista_debug_5)
                lista_debug_5_date.append(word_5_date)
        return lista_debug_5, lista_debug_5_date

In [ ]:

a = 0
for i in range(0,len(df)):
    link = df.iloc[i,1]
    date = df.iloc[i,2]
    html = urlopen(link).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    
    str_soup = str(soup)
    
    if str_soup.__contains__('Mais Gasolina'):
        
    
        if str_soup[0:13]=="Mais Gasolina":
            
            if str_soup.__contains__('Preços de referência'):
                g = 1
                str_soup_1 = str_soup.split()               #Passar para lista, retirando os espaço entre palavras
                filtered_1 = remove_words(str_soup_1)       #Remover stopwords   
                fuel_soup_1 = fuel(g,filtered_1)            #Extrair possiveis preços
                list_1.extend(fuel_soup_1)                  #Guardar preços
                list_dates_1.append(date)                   #Guardar respectiva data do ficheiro 
                

            elif str_soup.__contains__("Comparador Galp / BP / Repsol"):
                str_soup_2 = str_soup.split()               
                a+=1
                g = 2            
                ###### Definir o índice onde começa e onde termina
                start_index = str_soup_2.index('Galp')
                end_index = 78+start_index
                str_soup_2 = str_soup_2[start_index:end_index]
                # Extrair o subgrupo
                if str_soup_2[-1].endswith("operar"):       
                    filtered_2 = remove_words(str_soup_2)
                    fuel_soup_2 = fuel(g, filtered_2)
                    list_2.extend(fuel_soup_2)
                    list_dates_2.append(date)
                    
                else:                 
                    g = 3
                    filtered_2a = remove_words(str_soup_2)
                    fuel_soup_2a = fuel(g, filtered_2a)
                    list_2.extend(fuel_soup_2a)
                    list_dates_2.append(date)
                    
            else:          
                if str_soup.__contains__('Os mais baratos'):
                    #print(i)   
                    #print(str_soup)
                    #print()
                    print("------------------------------------------3----------------------------------------------------- \n")
                    #print(link)
                    str_soup_3 = str_soup.split()
                    #print(str_soup_3)
                    #filtered = remove_words(str_soup)
                    #list_3.extend(filtered)
    else:
        
        if str_soup.__contains__('Gasóleo colorido') or str_soup.__contains__('Gasóleo especial') or str_soup.__contains__('Gasóleo simples') or str_soup.__contains__('Gasolina especial 98') or str_soup.__contains__('Gasolina simples 95'):
            str_soup_4 = str_soup.split()
            g = 4
            filtered_4 = remove_words(str_soup_4)
            start_index_4 = filtered_4.index('Última')
            end_index_4 = start_index + 50
            filtered_4 = filtered_4[start_index_4:end_index_4]
            fuel_soup_4, fuel_soup_4_date= fuel(g, filtered_4)
            list_4.extend(fuel_soup_4)
            list_dates_4.extend(fuel_soup_4_date)
        else:
            print("------------------------------------------5----------------------------------------------------- \n")
            str_soup_5 = str_soup.split()
            g = 5
            filtered_5 = remove_words(str_soup_5)
            start_index_5 = filtered_5.index('Última')
            end_index_5 = start_index_5 + 50
            filtered_5 = filtered_5[start_index_5:end_index_5]
            fuel_soup_5, fuel_soup_5_date = fuel(g, filtered_5)
            list_5.extend(fuel_soup_5)
            list_dates_5.extend(fuel_soup_5_date)
            




print(f"Dates 1:", list_dates_1)
print(f"\nlista 1 :{list_1}\n")

print(f"Dates 2:",list_dates_2)
print(f"\nlista 2 :{list_2}\n")


list_comp = list_dates_4[:]
list_comp_price = list_4[:]
seen_price = set()
seen_date = set()
d = p = 0

for price, date in zip(list_4, list_dates_4):
    if date in seen_date and price in seen_price:
        list_comp.remove(date)
        list_comp_price.remove(price)

    elif date in seen_date and price not in seen_price:
        pass

    elif date not in seen_date and price in seen_price:
        pass

    else:
        seen_date.add(date)
        seen_price.add(price)

list_dates_4 = list_comp
list_4 = list_comp_price

for i in range(len(list_4)):
    word = list_4[i]
    word = word.replace(",",".")
    list_4[i] = float(word)

print(f"lista 4 :{list_4}\n")
print(f"Date 4 :{list_dates_4}\n")
#print(len(list_4)%len(list_dates_4) == 0)

for i in range(len(list_5)):
    word = list_5[i]
    word = word.replace(",",".")
    list_5[i] = float(word)

print(f"lista 5 :{list_5}\n")
print(f"Date 5 :{list_dates_5}\n")
#print(len(list_5)%len(list_dates_5) == 0)

for n in range(1,6):
    final_list.extend(eval(f"list_{n}"))
    final_list_dates.extend(eval(f"list_dates_{n}"))
print(f"Final list: \n", final_list)
print(f"\nFinal list for dates: \n", final_list_dates) 
#print(f"Verification: ", len(final_list)%len(final_list_dates)==0 )

ziped = list(zip(final_list_dates, final_list))
ziped.sort()
final_list_dates, final_list = zip(*ziped)
final_list_dates = list(final_list_dates)
final_list = list(final_list)

dict_price = {'Price': final_list, 
              'Date': final_list_dates}

print(f"\nFinal :",dict_price)

f.close()

médio
do
10
Mai
de
Preço
2011
Preço
e
descida
histórico,
e
Jan
Gasóleo.
subir
até
cent/litro
gasóleo
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
em
['2008-02-18', '2008-10-22', '2008-02-18', '2008-10-22']
lista 1 :['1.374', '1.368', '1.374', '1.359', '1.359', '1.359', '1.374', '1.368', '1.374', '1.359', '1.359', '1.359']

lista 2 :['1.569', '1.589', '1.574', '1.569', '1.589', '1.574', '1.584', '1.599', '1.589', '1.624', '1.644', '1.619', '1.664', '1.684', '1.664', '1.654', '1.674', '1.654', '1.649', '1.674', '1.654', '1.634', '1.654', '1.634', '1.619', '1.639', '1.624', '1.644', '1.674', '1.649', '1.619', '1.649', '1.634', '1.463', '1.484', '1.474', '1.489', '1.494', '1.489', '1.524', '1.544', '1.534', '1.539', '1.559', '1.544', '1.594', '1.614', '1.594', '1.629', '1.649', '1.629', '1.659', '1.679', '1.659', '1.564', '1.579', '1.564', '1.589', '1.609', '1.594', '1.594', '1.614', '1.599', '1.584', '1.609', '1.579', '1.564', '1.599', '1

<div class="alert alert-block alert-info">
<h3>Future Steps</h3>
</div>

- Finish this step of data filtering
- Analyse the taxation's impact on fuel's final price
- Ploting the retrieved data
- Use ML tools to predict the future fuel price